In [1]:
!pip install pandas numpy scikit-learn matplotlib seaborn imbalanced-learn flask shap ipywidgets

In [ ]:
# Load and Prepare Data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Load dataset
df = pd.read_csv('predictive_maintenance.csv')

# Display first 5 rows
display(df.head())

# Basic preprocessing
X = df.drop('failure_status', axis=1)
y = df['failure_status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Handle class imbalance
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_res)
X_test_scaled = scaler.transform(X_test)

In [4]:
# Training and Saving the Model
from sklearn.ensemble import RandomForestClassifier
import joblib

# Initialize and train model+
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train_res)

# Evaluate
from sklearn.metrics import classification_report
y_pred = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred))

# Save model and scaler
joblib.dump(model, 'model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("Model saved!")

              precision    recall  f1-score   support

           0       0.90      0.88      0.89        42
           1       0.99      0.99      0.99       358

    accuracy                           0.98       400
   macro avg       0.94      0.93      0.94       400
weighted avg       0.98      0.98      0.98       400

Model saved!


In [7]:
#creating prediction function
def predict_failure(temperature, vibration, power_usage, hours_used):
    """Predict washing machine failure probability."""
    
    # Load model and scaler
    model = joblib.load('model.pkl')
    scaler = joblib.load('scaler.pkl')
    
    # Create input array
    X_new = np.array([[temperature, vibration, power_usage, hours_used]])
    
    # Scale features
    X_new_scaled = scaler.transform(X_new)
    
    # Make prediction
    pred = model.predict(X_new_scaled)[0]
    proba = model.predict_proba(X_new_scaled)[0][1]
    
    # Return results
    result = {
        'prediction': 'Failure Likely' if pred == 1 else 'Normal',
        'probability': f"{proba:.1%}",
        'temperature': temperature,
        'vibration': vibration
    }
    return result

In [3]:
from ipywidgets import interact, FloatSlider
import matplotlib.pyplot as plt
import numpy as np
import joblib

# Load your model and scaler
model = joblib.load('model.pkl')
scaler = joblib.load('scaler.pkl')

def predict_failure(temperature, vibration, power_usage, hours_used):
    """Predict using your actual model"""
    X_new = np.array([[temperature, vibration, power_usage, hours_used]])
    X_new_scaled = scaler.transform(X_new)
    pred = model.predict(X_new_scaled)[0]
    proba = model.predict_proba(X_new_scaled)[0][1]
    return {
        'prediction': 'Failure Likely' if pred == 1 else 'Normal',
        'probability': f"{proba:.1%}"
    }

def interactive_prediction(temperature, vibration, power_usage, hours_used):
    result = predict_failure(temperature, vibration, power_usage, hours_used)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Gauge chart - CORRECTED SYNTAX
    prob = float(result['probability'].strip('%'))/100
    ax1.pie(
        [prob, 1-prob],
        labels=['Failure', 'Normal'],
        colors=['#ff6b6b', '#51cf66'],
        startangle=90,
        wedgeprops={'width': 0.3}  # Changed from dict(width=0.3) to avoid syntax issues
    )
    ax1.text(0, 0, result['probability'], ha='center', va='center', fontsize=20)
    ax1.set_title('Failure Probability', pad=20)
    
    # Feature indicators
    features = {
        'Temperature': temperature,
        'Vibration': vibration,
        'Power Usage': power_usage,
        'Hours Used': hours_used
    }
    ax2.barh(list(features.keys()), list(features.values()), color='#4cc9f0')
    ax2.set_title('Input Features')
    ax2.set_xlim(0, max(features.values())*1.1)
    
    plt.tight_layout()
    plt.show()
    print(f"Final Prediction: {result['prediction']} ({result['probability']} probability)")

# Now run the widget
interact(
    interactive_prediction,
    temperature=FloatSlider(min=20, max=80, step=1, value=45),
    vibration=FloatSlider(min=0, max=10, step=0.1, value=3),
    power_usage=FloatSlider(min=200, max=800, step=10, value=500),
    hours_used=FloatSlider(min=0, max=5000, step=100, value=2000)
)

interactive(children=(FloatSlider(value=45.0, description='temperature', max=80.0, min=20.0, step=1.0), FloatS…

<function __main__.interactive_prediction(temperature, vibration, power_usage, hours_used)>

In [3]:
# Save both model and scaler to files (run this in Jupyter)
import joblib

joblib.dump(model, 'model.pkl')  #trained model
joblib.dump(scaler, 'scaler.pkl')  #scaler

print("Model files saved!")

NameError: name 'model' is not defined